In [1]:
import pandas as pd
import numpy as np
import glob
import os

def load_stock_data(file_path):
    """
    Loads and processes individual stock CSV maintaining original structure.
    Handles assets with different historical periods.
    """
    try:
        # Read CSV without skipfooter, just skip header
        df = pd.read_csv(file_path, sep=';', decimal=',', thousands='.', 
                        encoding='utf-8', skiprows=12)
        
        # Remove empty rows and columns more precisely
        df = df.replace(['', '%'], np.nan).infer_objects(copy=False)
        df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
        
        # Convert dates first to ensure proper alignment
        month_map = {
            'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6,
            'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12
        }
        
        def convert_date(date_str):
            try:
                if pd.isna(date_str):
                    return None
                month, year = str(date_str).lower().strip().split('-')
                month_num = month_map.get(month.strip(), 1)
                return pd.Timestamp(2000 + int(year), month_num, 1)
            except Exception as e:
                print(f"Error converting date {date_str}: {str(e)}")
                return None
        
        # Convert dates and drop rows with invalid dates
        df['Dates'] = df.iloc[:,0].apply(convert_date)
        df = df.dropna(subset=['Dates'])
        
        # Set dates as index before processing other columns
        df.set_index('Dates', inplace=True)
        
        # Define column mappings with proper type conversion
        cols = {
            'PX_OPEN': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'PX_HIGH': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'PX_LOW': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'PX_LAST': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'PX_VOLUME': pd.to_numeric(df.iloc[:,2], errors='coerce'),
            'PX_TO_BOOK_RATIO': pd.to_numeric(df.iloc[:,7], errors='coerce'),
            'BS_CUR_LIAB': pd.to_numeric(df.iloc[:,25], errors='coerce'),
            'CUR_MKT_CAP': pd.to_numeric(df.iloc[:,2], errors='coerce'),
            'EBITDA': pd.to_numeric(df.iloc[:,17], errors='coerce'),
            'NET_INCOME': pd.to_numeric(df.iloc[:,18], errors='coerce'),
            'PE_RATIO': pd.to_numeric(df.iloc[:,6], errors='coerce'),
            'EV_EBITDA': pd.to_numeric(df.iloc[:,8], errors='coerce'),
            'DEBT_TO_EBITDA': pd.to_numeric(df.iloc[:,30], errors='coerce'),
            'GROSS_MARGIN': pd.to_numeric(df.iloc[:,14], errors='coerce'),
            'EBIT_MARGIN': pd.to_numeric(df.iloc[:,15], errors='coerce'),
            'LIQ_RATIO': pd.to_numeric(df.iloc[:,29], errors='coerce'),
            'ROE': pd.to_numeric(df.iloc[:,18], errors='coerce'),
            'ROA': pd.to_numeric(df.iloc[:,18], errors='coerce') / pd.to_numeric(df.iloc[:,24], errors='coerce'),
            'ASSET_TURNOVER': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'CAPEX': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'FCF': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'ROIC': pd.to_numeric(df.iloc[:,1], errors='coerce'),
            'NET_DEBT': pd.to_numeric(df.iloc[:,1], errors='coerce')
        }
        
        # Create DataFrame using the processed index
        processed_df = pd.DataFrame(cols, index=df.index)
        
        # Identificar o período válido do ativo
        # Consideramos como início a primeira data com pelo menos um valor não-NA
        first_valid = processed_df.apply(lambda x: x.first_valid_index()).min()
        last_valid = processed_df.apply(lambda x: x.last_valid_index()).max()
        
        if first_valid is None or last_valid is None:
            ticker = os.path.basename(file_path).replace('.csv', '')
            print(f"Excluindo {ticker}: Nenhum dado válido encontrado")
            return None
        
        # Recortar o DataFrame para incluir apenas o período com dados
        processed_df = processed_df[first_valid:last_valid]
        
        # Calcular a porcentagem de valores faltantes apenas no período válido
        missing_percentage = (processed_df.isna().sum().sum() / 
                            (processed_df.shape[0] * processed_df.shape[1])) * 100
        
        ticker = os.path.basename(file_path).replace('.csv', '')
        print(f"\nAnálise do ativo {ticker}:")
        print(f"Período de dados: {first_valid.strftime('%Y-%m-%d')} até {last_valid.strftime('%Y-%m-%d')}")
        print(f"Total de {(last_valid - first_valid).days / 365.25:.1f} anos de histórico")
        print(f"Porcentagem de valores faltantes: {missing_percentage:.2f}%")
        
        # Se tiver mais de 30% de valores faltantes no período válido, retorna None
        if missing_percentage > 30:
            print(f"Excluindo {ticker}: {missing_percentage:.2f}% de valores faltantes")
            return None
            
        # Tratamento de valores faltantes de forma mais segura
        # 1. Forward fill
        processed_df = processed_df.ffill()
        
        # 2. Backward fill
        processed_df = processed_df.bfill()
        
        # 3. Preenchimento com medianas - método atualizado
        for column in processed_df.columns:
            median_value = processed_df[column].median()
            if pd.isna(median_value):
                processed_df.loc[:, column] = processed_df[column].fillna(0)
            else:
                processed_df.loc[:, column] = processed_df[column].fillna(median_value)
        
        # 4. Validação final
        na_count = processed_df.isna().sum().sum()
        if na_count > 0:
            print(f"Warning: Found {na_count} NA values in {file_path}")
            # Último recurso: preencher NA restantes com 0
            fill_dict = {col: 0 for col in processed_df.columns if processed_df[col].isna().any()}
            processed_df = processed_df.fillna(fill_dict)
        
        # Agora sim fazemos o resample para daily
        processed_df = processed_df.resample('D').ffill()
        
        return processed_df
    
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        raise

# Resto do código permanece igual...

def create_merged_dataset(input_folder='./csv_files/', output_file='fundamentals.csv'):
    """
    Process all CSV files and create merged dataset compatible with load_data()
    """
    csv_files = glob.glob(os.path.join(input_folder, '*.csv'))
    all_data = {}
    
    if not csv_files:
        raise ValueError(f"No CSV files found in {input_folder}")
    
    print(f"Found {len(csv_files)} CSV files")
    
    # Process each file
    skipped_files = 0
    for file in csv_files:
        ticker = os.path.basename(file).replace('.csv', '')
        try:
            print(f"\nProcessando {ticker}...")
            df = load_stock_data(file)
            if df is not None and not df.empty:
                all_data[f"{ticker} Index"] = df
                print(f"Successfully processed {ticker} Index")
                print(f"DataFrame shape: {df.shape}")
                print(f"Date range: {df.index.min()} to {df.index.max()}")
            else:
                print(f"Ativo {ticker} excluído devido a dados insuficientes")
                skipped_files += 1
        except Exception as e:
            print(f"Error processing {ticker} Index: {str(e)}")
            skipped_files += 1
            continue
    
    if not all_data:
        raise ValueError("No data was successfully processed")
    
    print(f"\nResumo do processamento:")
    print(f"Total de arquivos encontrados: {len(csv_files)}")
    print(f"Arquivos processados com sucesso: {len(all_data)}")
    print(f"Arquivos excluídos: {skipped_files}")
    
    # Create proper MultiIndex DataFrame
    print("\nMerging all data...")
    merged_df = pd.concat(all_data, axis=1)
    
    # Ajustar o cabeçalho para incluir "Dates" na primeira coluna
    merged_df.reset_index(inplace=True)  # Mover 'Dates' (índice) para colunas

    # Configurar o cabeçalho com duas linhas
    first_header = ['Dates'] + [col[0] for col in merged_df.columns[1:]]
    second_header = ['Dates'] + [col[1] for col in merged_df.columns[1:]]
    
    # Salvar o CSV com duas linhas de cabeçalho
    with open(output_file, 'w', encoding='utf-8') as f:
        # Escrever primeira linha do cabeçalho
        f.write(','.join(first_header) + '\n')
        # Escrever segunda linha do cabeçalho
        f.write(','.join(second_header) + '\n')
        # Escrever os dados
        merged_df.to_csv(f, index=False, header=False)
    
    print(f"\nProcessed data saved to {output_file}")
    print(f"Total stocks processed: {len(all_data)}")
    
    return merged_df

In [2]:
# Então execute
merged_data = create_merged_dataset(
    input_folder='./csv_files/',  # pasta onde estão seus arquivos CSV
    output_file='fundamentals.csv'  # nome do arquivo de saída
)

Found 12 CSV files

Processando Vale3...

Análise do ativo Vale3:
Período de dados: 2019-12-01 até 2024-09-01
Total de 4.8 anos de histórico
Porcentagem de valores faltantes: 0.00%
Successfully processed Vale3 Index
DataFrame shape: (1737, 23)
Date range: 2019-12-01 00:00:00 to 2024-09-01 00:00:00

Processando ITUB4...
Error processing file ./csv_files/ITUB4.csv: single positional indexer is out-of-bounds
Error processing ITUB4 Index: single positional indexer is out-of-bounds

Processando BBAS3...
Error processing file ./csv_files/BBAS3.csv: single positional indexer is out-of-bounds
Error processing BBAS3 Index: single positional indexer is out-of-bounds

Processando SUZB3...

Análise do ativo SUZB3:
Período de dados: 2014-12-01 até 2024-09-01
Total de 9.8 anos de histórico
Porcentagem de valores faltantes: 0.00%
Successfully processed SUZB3 Index
DataFrame shape: (3563, 23)
Date range: 2014-12-01 00:00:00 to 2024-09-01 00:00:00

Processando B3SA3...

Análise do ativo B3SA3:
Período 